# Run a recon holdings in different scopes

You can run the cells below directly in LUSID's JupyterHub.

The `%%luminesce` is a magic command which passes the cell query string to Lumipy,
which then returns a DataFrame.
    

#### Step 1: Create transaction portfolios in two scopes

In [ ]:
%%luminesce

-- ====================================================
-- Description:
-- 1. In this query, we create Transaction Portfolios in LUSID
-- in two scopes
-- ===========================================================

-- Defining scope and code variables

@@file_date = select strftime('20221001');
@@base_currency = select 'GBP';
@@created_date = select #2000-01-01#;

@portfolios_from_excel = use Drive.Excel with @@file_date
--file=/luminesce-examples/equity_holdings_{@@file_date}.xlsx
--worksheet=portfolios
--addFileName
enduse;


-- Define the portfolio data

@create_portfolio =

select
'Transaction' as PortfolioType,
portfolio_scope as PortfolioScope,
portfolio_code as PortfolioCode,
portfolio_code as DisplayName,
portfolio_code as Description,
@@created_date as Created,
'' as SubHoldingKeys,
@@base_currency as BaseCurrency
from @portfolios_from_excel;

-- Upload the portfolio into LUSID

@response_create_portfolio =

select *
from Lusid.Portfolio.Writer
where ToWrite = @create_portfolio;

select *
from @response_create_portfolio;


#### Step 2: Create instruments

In [ ]:
%%luminesce

-- ====================================================
-- Description:
-- 1. In this query, we  load instruments from an Excel
-- file into LUSID
-- ====================================================

@@file_date =

select strftime('20221001');

@instruments_from_excel =

use Drive.Excel with @@file_date
--file=/luminesce-examples/equity_holdings_{@@file_date}.xlsx
--worksheet=instruments
--addFileName
enduse;

-- Run instruments transformation
@instruments_for_upload =

select Ticker, Name as DisplayName, ISIN as Isin, 'EQ' || Isin as ClientInternal, SEDOL as Sedol,
   Currency as DomCcy, 'Equities' as AssetClass, 'Equities' as SimpleInstrumentType
from @instruments_from_excel;

-- Upload the transformed data into LUSID
select *
from Lusid.Instrument.SimpleInstrument.Writer
where ToWrite = @instruments_for_upload;


#### Step 3: Upload abor transactions

In [ ]:
%%luminesce

-- ==================================================
-- Description:
-- 1. In this query, we load transactions into LUSID
-- ===================================================

@@file_date =

select strftime('20221001');

@@portfolioScope =

select 'abor-recon-test';

@transactions_from_excel =

use Drive.Excel with @@file_date
--file=/luminesce-examples/equity_holdings_{@@file_date}.xlsx
--worksheet=abor_transactions
--addFileName
enduse;

@txns =
select
'EQ' || InstrumentId as ClientInternal,
portfolio as PortfolioCode,
scope as PortfolioScope,
'GBP' as TradeCurrency,
'GBP' as SettlementCurrency,
#2022-02-01# as TransactionDate,
#2022-02-01# as SettlementDate,
price as TradePrice,
TxnId as TxnId,
transaction_type as Type,
total_consideration as TotalConsideration,
units as Units,
'Upsert' as WriteAction
from @transactions_from_excel;

-- step 3: load a transaction

@load_txn =
select * from Lusid.Portfolio.Txn.Writer
where ToWrite = @txns;

select * from @load_txn;


#### Step 4: Upload ibor holdings

In [ ]:
%%luminesce

-- ===============================================
-- Description:
-- 1. In this query, we load holdings into LUSID
-- ================================================

@@file_date =

select strftime('20221001');

@@portfolioScope =

select 'ibor-recon-test';

@holdings_from_excel =

use Drive.Excel with @@file_date
--file=/luminesce-examples/equity_holdings_{@@file_date}.xlsx
--worksheet=lusid_holdings
--addFileName
enduse;

-- Run holdings transformation
@holdings_for_upload =

select portfolio as [PortfolioCode], @@portfolioScope as [PortfolioScope], holding_date as
   [EffectiveAt], 'EQ' || InstrumentId as [ClientInternal], units as [Units], Currency as
   CostCurrency
from @holdings_from_excel;

-- Upload the transformed data into LUSID
select *
from Lusid.Portfolio.Holding.Writer
where ToWrite = @holdings_for_upload;


#### Step 5: Create reconciliation view

In [ ]:
%%luminesce

-- ============================================================
-- Description:
-- 1. In this query, we create a view to run the reconciliation
-- ============================================================


@recon_view =

use Sys.Admin.SetupView
--provider=Recon.IborVersusAbor

--parameters
aborScope,Text,abor-recon-test,true
iborScope,Text,ibor-recon-test,true
reconDate,Date,2022-03-01,true
----

@@aborScope = select #PARAMETERVALUE(aborScope);
@@iborScope = select #PARAMETERVALUE(iborScope);
@@reconDate = select #PARAMETERVALUE(reconDate);

-- Get ABOR holdings

@abor_data = select
h.PortfolioCode,
i.DisplayName as [InstrumentName],
i.LusidInstrumentId,
i.ClientInternal,
i.Isin,
h.HoldingType,
h.Units
from Lusid.Portfolio.Holding h
join Lusid.Instrument i on (i.LusidInstrumentId = h.LusidInstrumentId)
where h.PortfolioScope = @@aborScope
and h.EffectiveAt = @@reconDate
order by h.HoldingType desc;

-- Get IBOR holdings

@ibor_data = select
h.PortfolioCode,
i.DisplayName as [InstrumentName],
i.LusidInstrumentId,
i.ClientInternal,
i.Isin,
h.HoldingType,
h.Units
from Lusid.Portfolio.Holding h
join Lusid.Instrument i on (i.LusidInstrumentId = h.LusidInstrumentId)
where h.PortfolioScope = @@iborScope
and h.EffectiveAt = @@reconDate
order by h.HoldingType desc;

select
a.PortfolioCode as [Abor_PortfolioCode],
a.InstrumentName as [Abor_InstrumentName],
a.Isin as [Abor_Isin],
a.HoldingType as [Abor_HoldingType],
a.Units as [Abor_Units],
i.Units as [Ibor_Units],
(a.Units - i.Units) as [Units_Diff],
round(((a.Units *1.0) / i.Units) -1 ,5) as [Units_Diff_Pct],
case when (abs(a.Units - i.Units) > 0 or i.Units is null or a.Units is null) then 'Break' else 'NoBreak' end as [ReconStatus]
from @abor_data a
left join @ibor_data i on (a.ClientInternal = i.ClientInternal and i.PortfolioCode = a.PortfolioCode)
union
select
a.PortfolioCode as [Abor_PortfolioCode],
a.InstrumentName as [Abor_InstrumentName],
a.Isin as [Abor_Isin],
a.HoldingType as [Abor_HoldingType],
a.Units as [Abor_Units],
i.Units as [Ibor_Units],
(a.Units - i.Units) as [Units_Diff],
round(((a.Units *1.0) / i.Units) -1 ,5) as [Units_Diff_Pct],
case when (abs(a.Units - i.Units) > 0 or i.Units is null or a.Units is null) then 'Break' else 'NoBreak' end as [ReconStatus]
from @ibor_data i
left join @abor_data a on (a.ClientInternal = i.ClientInternal and i.PortfolioCode = a.PortfolioCode)
;

enduse;

select * from @recon_view wait 10;


#### Step 6: Run recon workflow

In [ ]:
%%luminesce

-- ===========================================
-- Description:
-- 1. In this query, we run the reconciliation
-- ===========================================

-- Define params

@@recon_date = select date(#2022-03-02#);
@@recon_date_str = select strftime('%Y%m%d', @@recon_date);
@@abor_scope = select 'abor-recon-test';
@@ibor_scope = select 'ibor-recon-test';

-- Run reconciliations

@recon_data = select * from Recon.IborVersusAbor
where reconDate = @@recon_date
and aborScope = @@abor_scope
and iborScope = @@ibor_scope;

@recon_passed = select * from @recon_data where ReconStatus = "NoBreak";
@recon_breaks = select * from @recon_data where ReconStatus = "Break";

-- Save results

@save_recon_passed_to_drive = use Drive.SaveAs with @recon_passed, @@recon_date_str
--path=/luminesce-examples/ReconciliationResults/ReconPassed
--fileNames=recon_passed_{@@recon_date_str}
enduse;

@save_recon_breaks_to_drive = use Drive.SaveAs with @recon_breaks, @@recon_date_str
--path=/luminesce-examples/ReconBreaks
--fileNames=recon_failed_{@@recon_date_str}
enduse;


select
FileName,
RowCount,
Skipped
from
@save_recon_passed_to_drive
union
select
FileName,
RowCount,
Skipped
from
@save_recon_breaks_to_drive;


#### Step 7: Run recon with generic reconciliation provider

In [ ]:
%%luminesce

-- =============================================================
-- Description:
-- 1. In this query, we perform a reconciliation on 2 portfolios
-- using the generic reconciliation provider
-- =============================================================

------------------------------
-- Table - Lookup to return --
------------------------------

-- (Required)

@lookup = values
('ibor-recon-test', 'ibor-recon-test', 'UkEquity001', 'SinglePortfolio', 'abor-recon-test', 'UkEquity001', 'SinglePortfolio', 'ibor-recon-test/default', 'ibor-recon-test/default', 'GBP', 'GBP', #2022-11-11#, #2022-11-11#)
;

@lookup_to_return =
select
    column1 as ReconciliationKey,
    column2 as LeftPortfolioScope,
    column3 as LeftPortfolioCode,
    column4 as LeftPortfolioType,
    column5 as RightPortfolioScope,
    column6 as RightPortfolioCode,
    column7 as RightPortfolioType,
    column8 as LeftRecipeId,
    column9 as RightRecipeId,
    column10 as LeftReportCurrency,
    column11 as RightReportCurrency,
    column12 as LeftValuationDate,
    column13 as RightValuationDate
from @lookup;

--------------------------------
-- Table - Measures to return --
--------------------------------

-- (Optional)

@measures = values
('Holding/default/Units', 'Sum', null),
('Holding/default/Cost', 'Sum', null)
;

@measures_to_return = select
    column1 as 'MeasureName',
    column2 as 'Operation',
    column3 as 'ReconciliationSide'
from @measures;

------------------------------
-- Table - Comparison Rules --
------------------------------

-- (Optional)

@comparisons = values
('ReconcileNumericRule', 'AbsoluteDifference', '200', 'Holding/default/Units', 'Sum')
;

@comparison_rules = select
    column1 as 'RuleType',
    column2 as 'ComparisonType',
    column3 as 'Tolerance',
    column4 as 'AppliesToKey',
    column5 as 'AppliesToOp'
from @comparisons;

----------------------------------
-- Reconciliation Provider Call --
----------------------------------

@rec_response = select * from Lusid.Portfolio.Reconciliation.Generic where
    MeasuresToReturn = @measures_to_return
    and
    ToLookup = @lookup_to_return
    and
    ComparisonRules = @comparison_rules
    and
    UseDefaultGroupKeys = false
    and
    KeysToGroupBy = 'Instrument/default/LusidInstrumentId'
    and
    Error is null
;


@pivot = use Tools.Pivot with @rec_response
--key=Measure
--aggregateColumns=LeftMeasureValue,RightMeasureValue,Difference,ResultComparison
enduse;

-- Uncomment one of the below 2 lines for either raw or pivoted output

-- Raw Output
select * from @rec_response;

-- Pivot Output
-- select * from @pivot;
